In [1]:
import numpy as np
import re
import feedparser

In [4]:
def TrainSet():
    postList=[['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    label = [0,1,0,1,0,1] #1 is abusive, 0 not
    return postList, label
def AllWords(postList):
    All_word = []
    for word_list in postList:
        All_word += word_list
    return set(All_word)
def CountNumWords(allWords, inputSet):
    count = np.zeros(len(allWords))
    for ips in inputSet:
        if ips in allWords:
            count[list(allWords).index(ips)] += 1
        else:
            print("{} is beyond our dataset".format(ips))
    return count
def NavieBayes(trainMat, label):
    num_all_words = len(trainMat[0])
    num_document = len(trainMat)
    p_priori_1 = np.log((sum(label) + 1) / (num_document + 2))
    p_priori_0 = np.log((num_document - sum(label) + 1) / (num_document + 2))
    p1 = np.ones(num_all_words)
    p0 = np.ones(num_all_words)
    p1_denominator = 2
    p0_denominator = 2
    for i in range(num_document):
        if label[i] == 1:
            p1 += trainMat[i]
            p1_denominator += sum(trainMat[i])
        else:
            p0 += trainMat[i]
            p0_denominator += sum(trainMat[i])
    p1 = np.log(p1 / p1_denominator)
    p0 = np.log(p0 / p0_denominator)
    return p1, p0, p_priori_1, p_priori_0
def DetermineCategory(countNumWords, p1, p0, pp1, pp0):
    if sum(countNumWords * p1) + pp1 > sum(countNumWords * p0) + pp0:
        return 1
    else:
        return 0
def ClassifierNaiveBayes():
    postList, lable = TrainSet()
    AW = AllWords(postList)
    trainMat = []
    for pl in postList:
        trainMat.append(CountNumWords(AW, pl))
    p1, p0, pp1, pp0 = NavieBayes(trainMat, lable)
    testEntry = ['love', 'my', 'dalmation']
    thisDoc = CountNumWords(AW, testEntry)
    print('This document is {}'.format(DetermineCategory(thisDoc, p1, p0, pp1, pp0)))
    testEntry = ['stupid', 'garbage']
    thisDoc = CountNumWords(AW, testEntry)
    print('This document is {}'.format(DetermineCategory(thisDoc, p1, p0, pp1, pp0)))
def TextParse(WholePage):
    WordsList = re.split('\W+', WholePage)
    return [word.lower() for word in WordsList if len(word)>=2]
def ListOfRandoms(num, rangeMin, rangeMax):
    ListRandom = []
    while(len(ListRandom) != num):
        r = int(np.random.uniform(rangeMin, rangeMax))
        if r not in ListRandom:
            ListRandom.append(r)
    return np.sort(ListRandom)
def SpamEmailDetect():
    docList = []
    label = []
    for i in range(1, 26):
        docList.append(TextParse(open('email/spam/{}.txt'.format(i)).read()))
        label.append(1)
        docList.append(TextParse(open('email/ham/{}.txt'.format(i)).read()))
        label.append(0)
    AW = AllWords(docList)
    testSet = []
    testLabel = []
    indexList = ListOfRandoms(10, 0, 50)
    for index in indexList[::-1]:
        testSet.append(docList[index])
        testLabel.append(label[index])
        del docList[index]
        del label[index]
    trainMat = []
    for doc in docList:
        trainMat.append(CountNumWords(AW, doc))
    p1, p0, pp1, pp0 = NavieBayes(trainMat, label)
    error = 0
    for i in range(len(testSet)):
        thisDoc = CountNumWords(AW, testSet[i])
        if DetermineCategory(thisDoc, p1, p0, pp1, pp0) != testLabel[i]:
            error = error + 1
    #print('Error rate is {}'.format(error/len(testSet)))
    return (error/len(testSet))

In [5]:
ClassifierNaiveBayes()

This document is 0
This document is 1


In [6]:
error = []
for i in range(100):
    error.append(SpamEmailDetect())
print('Mean error rate is {}'.format(np.mean(error)))

Mean error rate is 0.055999999999999994


In [12]:
ny = feedparser.parse('https://newyork.craigslist.org/stp/index.rss')

In [23]:
ny['feed']

{'html': {'class': 'no-js'},
 'links': [{'href': 'https://www.craigslist.org/styles/simple-page.css?v=1522ad26a5713653eb46d2c31b48f4fd',
   'media': 'all',
   'rel': 'stylesheet',
   'type': 'text/css'},
  {'href': 'https://www.craigslist.org/styles/jquery-ui-clcustom.css?v=3b05ddffb7c7f5b62066deff2dda9339',
   'media': 'all',
   'rel': 'stylesheet',
   'type': 'text/css'},
  {'href': 'https://www.craigslist.org/styles/jquery.qtip-2.2.1.css?v=cd202aead4d1dd4894fbae4ade23fcf8',
   'media': 'all',
   'rel': 'stylesheet',
   'type': 'text/css'}],
 'meta': {'content': 'width=device-width,initial-scale=1', 'name': 'viewport'}}

In [ ]:
def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print("SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print (item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])